In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from joblib import Parallel, delayed
import os
import logging
import time
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout

# Configuration
warnings.filterwarnings("ignore")
tf.keras.backend.set_floatx('float32')

CONFIG = {
    'forecast_horizon': 12,
    'seasonal_periods': 12,
    'min_data_length': 24,
    'img_dir': 'model_results',
    'results_file': 'model_results/model_results.csv',
    'n_jobs': 4,
    'outlier_threshold': 3,
    'max_diff': 3,
    'lag_features': list(range(1, 13)),
    'rolling_windows': [3, 6, 12],
    'correlation_threshold': 0.2,
    'dpi': 150,
    'sequence_length': 12, 
    'batch_size': 8,
    'epochs': 5,
}

# Setup logging
def setup_logging(img_dir):
    os.makedirs(img_dir, exist_ok=True)
    logging.basicConfig(
        filename=f'{img_dir}/forecast_log.txt',
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

# Outlier Detection
def detect_outliers(series, threshold=CONFIG['outlier_threshold']):
    logger = logging.getLogger(__name__)
    z_scores = np.abs((series - series.mean()) / series.std())
    outliers = z_scores > threshold
    series_clean = series.copy()
    # series_clean[outliers] = series.mean()
    logger.info(f"Detected {outliers.sum()} outliers")
    return series_clean

# Feature Engineering
def create_features(data, target, exog_cols, config=CONFIG):
    logger = logging.getLogger(__name__)
    logger.info(f"Creating features for {target}, data shape: {data.shape}")
    
    if len(data) < config['min_data_length']:
        raise ValueError(f"Data too short: {len(data)} rows")
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be DatetimeIndex")
    
    if target not in data.columns:
        raise ValueError(f"Target column {target} not found in data")
    
    df = data.copy()
    required_cols = [target] + exog_cols
    
    # Handle missing/infinite values
    if df.isna().any().any() or np.isinf(df).any().any():
        logger.warning("Data contains NaN or Inf, filling...")
        df = df.fillna(method='ffill').fillna(df.mean(numeric_only=True))
    
    # Vectorized lag and rolling features
    lag_features = {f'{target}_lag_{lag}': df[target].shift(lag) for lag in config['lag_features']}
    rolling_features = {
        f'{target}_roll_{stat}_{window}': getattr(df[target].rolling(window=window), stat)()
        for window in config['rolling_windows']
        for stat in ['mean', 'std']
    }
    df = df.assign(**lag_features, **rolling_features)
    
    # Add seasonal features
    df = df.assign(
        month=df.index.month,
        month_sin=np.sin(2 * np.pi * df.index.month / config['seasonal_periods']),
        month_cos=np.cos(2 * np.pi * df.index.month / config['seasonal_periods']),
        quarter=df.index.quarter
    )
    df = pd.get_dummies(df, columns=['month'], prefix='month')
    
    # Remove low-correlation features
    try:
        correlations = df.corr(numeric_only=True)[target].drop(required_cols, errors='ignore')
        low_corr_cols = correlations[abs(correlations) < config['correlation_threshold']].index
        logger.info(f"Low-correlation features: {list(low_corr_cols)}")
        if low_corr_cols.any():
            logger.info(f"Dropping {len(low_corr_cols)} low-correlation features")
            df = df.drop(columns=low_corr_cols)
    except Exception as e:
        logger.error(f"Error calculating correlations: {str(e)}")
    
    # Kiểm tra NaN sau khi tạo features
    if df.isna().any().any() or np.isinf(df).any().any():
        logger.warning("NaN/Inf in features after creation, filling...")
        df = df.fillna(df.mean(numeric_only=True))
    
    return df

# Stationarity Check
def check_stationarity(series, name, max_diff=CONFIG['max_diff']):
    logger = logging.getLogger(__name__)
    series_clean = series.dropna().replace([np.inf, -np.inf], np.nan).dropna()
    if len(series_clean) < 2:
        logger.error(f"{name}: Data too short after cleaning!")
        return 0
    
    d = 0
    while d <= max_diff:
        result = adfuller(series_clean)
        if result[1] < 0.05:
            logger.info(f"{name} stationary at d={d}")
            return d
        if d == max_diff:
            logger.warning(f"{name} not stationary after {max_diff} differencing")
            return d
        series_clean = series_clean.diff().dropna()
        if len(series_clean) < 2:
            logger.warning(f"{name}: Data too short after differencing {d+1}")
            return d
        d += 1
    return d

# Model Evaluation Metrics
def calculate_metrics(actual, predicted):
    logger = logging.getLogger(__name__)
    actual = np.array(actual, dtype=float)
    predicted = np.array(predicted, dtype=float)
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted) & ~np.isinf(actual) & ~np.isinf(predicted)
    actual = actual[valid_mask]
    predicted = predicted[valid_mask]
    
    if len(actual) == 0:
        logger.warning("No valid data for metrics calculation!")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted) * 100 if np.all(np.abs(actual) > 1e-8) else np.nan
    smape = 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    norm_mape = mape / np.mean(np.abs(actual)) if not np.isnan(mape) else np.nan
    directional_acc = np.mean((np.diff(actual) * np.diff(predicted)) > 0) * 100 if len(actual) > 1 else np.nan
    return rmse, mae, mape, smape, norm_mape, directional_acc

# Visualization Functions
def plot_decomposition(series, period, filename, img_dir=CONFIG['img_dir']):
    logger = logging.getLogger(__name__)
    if series.isna().any():
        series = series.fillna(method='ffill').fillna(series.mean())
    
    try:
        decomposition = seasonal_decompose(series, period=period, model='additive')
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 8))
        ax1.plot(series.index, series, label='Original'); ax1.legend(loc='upper left')
        ax2.plot(series.index, decomposition.trend, label='Trend'); ax2.legend(loc='upper left')
        ax3.plot(series.index, decomposition.seasonal, label='Seasonal'); ax3.legend(loc='upper left')
        ax4.plot(series.index, decomposition.resid, label='Residual'); ax4.legend(loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename), dpi=CONFIG['dpi'])
        logger.info(f"Saved decomposition plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving decomposition plot: {str(e)}")
    finally:
        plt.close()

def plot_forecast(historical, test, forecast, forecast_index, title, ylabel, filename, confidence_intervals=None, img_dir=CONFIG['img_dir']):
    logger = logging.getLogger(__name__)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(historical.index, historical, label='Historical', color='blue')
    ax.plot(test.index, test, label='Actual (Test)', color='green')
    ax.plot(forecast_index, forecast, label='Forecast', color='orange', linestyle='--', linewidth=2)
    if confidence_intervals:
        ax.fill_between(forecast_index, confidence_intervals[0], confidence_intervals[1], 
                        color='orange', alpha=0.2, label='95% CI')
    ax.set_title(title); ax.set_xlabel('Time'); ax.set_ylabel(ylabel); ax.legend(); ax.grid(True)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45); plt.tight_layout()
    plt.savefig(os.path.join(img_dir, filename), dpi=CONFIG['dpi'])
    plt.close()

def plot_comparison_forecasts(historical, test, forecasts, forecast_index, title, ylabel, filename, metrics=None, img_dir=CONFIG['img_dir']):
    logger = logging.getLogger(__name__)
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.plot(historical.index, historical, label='Historical', color='blue')
    ax.plot(test.index, test, label='Actual (Test)', color='green')
    colors = sns.color_palette("husl", len(forecasts))
    for (model_name, forecast), color in zip(forecasts.items(), colors):
        rmse = metrics.get(model_name, {}).get('RMSE', np.nan) if metrics else np.nan
        if forecast is None or pd.isna(rmse):
            continue
        ax.plot(forecast_index, forecast, label=f'{model_name} (RMSE: {rmse:.4f})', 
                linestyle='--', color=color)
    ax.set_title(title); ax.set_xlabel('Time'); ax.set_ylabel(ylabel); ax.legend(); ax.grid(True)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45); plt.tight_layout()
    plt.savefig(os.path.join(img_dir, filename), dpi=CONFIG['dpi'])
    plt.close()

def plot_metrics_bar(metrics_df, filename, img_dir=CONFIG['img_dir']):
    logger = logging.getLogger(__name__)
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    metrics = ['RMSE', 'MAE', 'MAPE', 'sMAPE', 'NormMAPE', 'DirAcc']
    for i, (metric, ax) in enumerate(zip(metrics, axes.flatten())):
        sns.barplot(x='Model', y=metric, data=metrics_df, ax=ax)
        ax.set_title(f'{metric} Comparison')
        ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(img_dir, filename), dpi=CONFIG['dpi'])
    plt.close()

def plot_residual_acf(residuals, title, filename, img_dir=CONFIG['img_dir']):
    logger = logging.getLogger(__name__)
    if residuals is None or len(residuals) < 2:
        logger.warning(f"Skipping ACF: Insufficient residual data - {title}")
        return
    fig, ax = plt.subplots(figsize=(5, 3))
    max_lags = min(20, len(residuals) - 1)
    if max_lags < 1:
        return
    plot_acf(residuals, lags=max_lags, title=title, ax=ax)
    plt.tight_layout()
    plt.savefig(os.path.join(img_dir, filename), dpi=CONFIG['dpi'])
    plt.close()

# Sequence Creation for LSTM/GRU
def create_sequences(data, seq_length, target_col):
    logger = logging.getLogger(__name__)
    data = data.copy()
    if data.isna().any().any():
        logger.warning("Data contains NaN in create_sequences, filling...")
        data = data.fillna(data.mean(numeric_only=True))
    if np.isinf(data).any().any():
        logger.warning("Data contains Inf in create_sequences, replacing...")
        data = data.replace([np.inf, -np.inf], data.mean(numeric_only=True))
    
    logger.info(f"Data head in create_sequences:\n{data.head()}")
    X, y = [], []
    if len(data) < seq_length + 1:
        logger.error(f"Data too short for sequence length {seq_length}: {len(data)}")
        return np.array([]), np.array([])
    
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length][[target_col]].values
        if np.isnan(seq).any() or np.isinf(seq).any():
            logger.warning(f"NaN/Inf in sequence {i}, skipping...")
            continue
        X.append(seq)
        y.append(data.iloc[i+seq_length][target_col])
    
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    if X.shape[0] == 0:
        logger.error("No valid sequences created")
    else:
        logger.info(f"Created {X.shape[0]} sequences with shape {X.shape}")
    return X, y

# Model Functions
def run_exponential_smoothing(train, test, forecast_index, seasonal_periods=CONFIG['seasonal_periods']):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        model = ExponentialSmoothing(train, trend='add', seasonal='add', 
                                  seasonal_periods=seasonal_periods).fit(optimized=True)
        forecast = model.forecast(CONFIG['forecast_horizon'])
        residuals = train - model.fittedvalues
        forecast = pd.Series(forecast.values, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"Exponential Smoothing forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        resid_std = np.std(residuals)
        ci_lower = forecast - 1.96 * resid_std
        ci_upper = forecast + 1.96 * resid_std
        metrics = calculate_metrics(test, forecast)
        logger.info(f"Exponential Smoothing: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Error Exponential Smoothing: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_sarima(train, test, forecast_index, exog_train=None, exog_test=None, seasonal_periods=CONFIG['seasonal_periods']):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        if exog_train is not None and exog_test is not None:
            exog_train = exog_train.fillna(exog_train.mean(numeric_only=True))
            exog_test = exog_test.fillna(exog_test.mean(numeric_only=True))
            if len(exog_test) < CONFIG['forecast_horizon']:
                logger.error(f"Exogenous test data too short: {len(exog_test)} < {CONFIG['forecast_horizon']}")
                return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        d = check_stationarity(train, "SARIMA")
        try:
            model = pm.auto_arima(train, exogenous=exog_train, start_p=0, start_q=0, max_p=5, max_q=5, d=d, max_d=1,
                                seasonal=True, m=seasonal_periods, start_P=0, start_Q=0, max_P=3, max_Q=3, max_D=1,
                                stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                                information_criterion='aic', maxiter=30)
        except:
            logger.warning("auto_arima failed, using SARIMA(1,1,1)(1,1,1,12)")
            model = pm.ARIMA(order=(1,1,1), seasonal_order=(1,1,1,seasonal_periods), 
                           suppress_warnings=True).fit(train, exogenous=exog_train)
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'], exogenous=exog_test)
        residuals = train - model.predict_in_sample(exogenous=exog_train)
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"SARIMA forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"SARIMA: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error SARIMA: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_prophet(train, test, forecast_index, exog_train=None, exog_future=None):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
        if exog_train is not None:
            for col in exog_train.columns:
                df_train[col] = exog_train[col].values
        if exog_future is not None:
            exog_future = exog_future.fillna(exog_future.mean(numeric_only=True))
            if len(exog_future) < CONFIG['forecast_horizon']:
                logger.error(f"Exogenous future data too short: {len(exog_future)} < {CONFIG['forecast_horizon']}")
                return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False,
                       changepoint_prior_scale=0.01, n_changepoints=10, seasonality_prior_scale=10.0)
        if exog_train is not None:
            for col in exog_train.columns:
                model.add_regressor(col)
        model.fit(df_train)
        future = pd.DataFrame({'ds': forecast_index})
        if exog_future is not None:
            for col in exog_future.columns:
                future[col] = exog_future[col].values
        forecast = model.predict(future)
        forecast_series = pd.Series(forecast['yhat'].values, index=forecast_index)
        if len(forecast_series) != CONFIG['forecast_horizon']:
            logger.error(f"Prophet forecast length {len(forecast_series)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        residuals = train - model.predict(df_train)['yhat']
        ci_lower = pd.Series(forecast['yhat_lower'].values, index=forecast_index)
        ci_upper = pd.Series(forecast['yhat_upper'].values, index=forecast_index)
        metrics = calculate_metrics(test, forecast_series)
        logger.info(f"Prophet: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast_series, residuals, metrics, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Error Prophet: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_xgboost(train, test, forecast_index, train_X, test_X):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        if train_X.isna().any().any() or test_X.isna().any().any():
            logger.warning("NaN in train_X or test_X, filling...")
            train_X = train_X.fillna(train_X.mean(numeric_only=True))
            test_X = test_X.fillna(test_X.mean(numeric_only=True))
        if np.isinf(train_X).any().any() or np.isinf(test_X).any().any():
            logger.warning("Inf in train_X or test_X, replacing...")
            train_X = train_X.replace([np.inf, -np.inf], train_X.mean(numeric_only=True))
            test_X = test_X.replace([np.inf, -np.inf], test_X.mean(numeric_only=True))
        if len(test_X) < CONFIG['forecast_horizon']:
            logger.error(f"Test features too short: {len(test_X)} < {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        model.fit(train_X, train)
        forecast = model.predict(test_X)
        residuals = train - model.predict(train_X)
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"XGBoost forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"XGBoost: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error XGBoost: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_lightgbm(train, test, forecast_index, train_X, test_X):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        if train_X.isna().any().any() or test_X.isna().any().any():
            logger.warning("NaN in train_X or test_X, filling...")
            train_X = train_X.fillna(train_X.mean(numeric_only=True))
            test_X = test_X.fillna(test_X.mean(numeric_only=True))
        if np.isinf(train_X).any().any() or np.isinf(test_X).any().any():
            logger.warning("Inf in train_X or test_X, replacing...")
            train_X = train_X.replace([np.inf, -np.inf], train_X.mean(numeric_only=True))
            test_X = test_X.replace([np.inf, -np.inf], test_X.mean(numeric_only=True))
        if len(test_X) < CONFIG['forecast_horizon']:
            logger.error(f"Test features too short: {len(test_X)} < {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        model = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        model.fit(train_X, train)
        forecast = model.predict(test_X)
        residuals = train - model.predict(train_X)
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"LightGBM forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"LightGBM: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error LightGBM: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_svr(train, test, forecast_index, train_X, test_X):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        if train_X.isna().any().any() or test_X.isna().any().any():
            logger.warning("NaN in train_X or test_X, filling...")
            train_X = train_X.fillna(train_X.mean(numeric_only=True))
            test_X = test_X.fillna(test_X.mean(numeric_only=True))
        if np.isinf(train_X).any().any() or np.isinf(test_X).any().any():
            logger.warning("Inf in train_X or test_X, replacing...")
            train_X = train_X.replace([np.inf, -np.inf], train_X.mean(numeric_only=True))
            test_X = test_X.replace([np.inf, -np.inf], test_X.mean(numeric_only=True))
        if len(test_X) < CONFIG['forecast_horizon']:
            logger.error(f"Test features too short: {len(test_X)} < {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        model = SVR(C=1, epsilon=0.1, kernel='rbf')
        scaler = MinMaxScaler()
        train_X_scaled = scaler.fit_transform(train_X)
        test_X_scaled = scaler.transform(test_X)
        if np.isnan(train_X_scaled).any() or np.isnan(test_X_scaled).any():
            logger.error("NaN in scaled data for SVR")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        model.fit(train_X_scaled, train)
        forecast = model.predict(test_X_scaled)
        residuals = train - model.predict(train_X_scaled)
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"SVR forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"SVR: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error SVR: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_lstm(train, test, forecast_index, train_X, test_X, seq_length=CONFIG['sequence_length']):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        # Chỉ sử dụng target (cpi_mom) để đơn giản hóa
        combined_data = pd.DataFrame({
            'target': pd.concat([train, test], axis=0)
        })
        logger.info(f"LSTM: combined_data shape: {combined_data.shape}")
        logger.info(f"LSTM: combined_data head:\n{combined_data.head()}")
        
        if combined_data.isna().any().any() or np.isinf(combined_data).any().any():
            logger.warning("NaN/Inf in combined_data, filling...")
            combined_data = combined_data.fillna(combined_data.mean(numeric_only=True))
        
        scaler = MinMaxScaler()
        combined_scaled = scaler.fit_transform(combined_data)
        if np.isnan(combined_scaled).any() or np.isinf(combined_scaled).any():
            logger.error("NaN/Inf in scaled data for LSTM")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        combined_scaled_df = pd.DataFrame(combined_scaled, index=combined_data.index, 
                                        columns=['target'])
        
        X_seq, y_seq = create_sequences(combined_scaled_df, seq_length, 'target')
        if X_seq.shape[0] == 0:
            logger.error("Failed to create sequences for LSTM")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        train_seq_len = len(train) - seq_length
        if train_seq_len <= 0:
            logger.error(f"Train sequence length too short: {train_seq_len}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        X_train_seq, y_train_seq = X_seq[:train_seq_len], y_seq[:train_seq_len]
        X_test_seq, y_test_seq = X_seq[train_seq_len:], y_seq[train_seq_len:]
        logger.info(f"LSTM: X_train_seq shape {X_train_seq.shape}, X_test_seq shape {X_test_seq.shape}")
        
        model = Sequential([
            LSTM(32, activation='relu', input_shape=(seq_length, 1)),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_seq, y_train_seq, epochs=CONFIG['epochs'], batch_size=CONFIG['batch_size'], verbose=0)
        
        # Recursive forecasting
        forecast = []
        input_seq = X_seq[train_seq_len - 1].copy()
        for i in range(CONFIG['forecast_horizon']):
            input_seq_reshaped = input_seq.reshape((1, seq_length, 1))
            pred = model.predict(input_seq_reshaped, verbose=0)[0, 0]
            forecast.append(pred)
            input_seq = np.roll(input_seq, -1, axis=0)
            input_seq[-1] = pred
        
        forecast_unscaled = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()
        residuals = train.iloc[seq_length:] - scaler.inverse_transform(
            model.predict(X_train_seq)[:, 0].reshape(-1, 1)
        ).flatten()
        forecast = pd.Series(forecast_unscaled, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"LSTM forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"LSTM: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error LSTM: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_gru(train, test, forecast_index, train_X, test_X, seq_length=CONFIG['sequence_length']):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        # Chỉ sử dụng target (cpi_mom) để đơn giản hóa
        combined_data = pd.DataFrame({
            'target': pd.concat([train, test], axis=0)
        })
        logger.info(f"GRU: combined_data shape: {combined_data.shape}")
        logger.info(f"GRU: combined_data head:\n{combined_data.head()}")
        
        if combined_data.isna().any().any() or np.isinf(combined_data).any().any():
            logger.warning("NaN/Inf in combined_data, filling...")
            combined_data = combined_data.fillna(combined_data.mean(numeric_only=True))
        
        scaler = MinMaxScaler()
        combined_scaled = scaler.fit_transform(combined_data)
        if np.isnan(combined_scaled).any() or np.isinf(combined_scaled).any():
            logger.error("NaN/Inf in scaled data for GRU")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        combined_scaled_df = pd.DataFrame(combined_scaled, index=combined_data.index, 
                                        columns=['target'])
        
        X_seq, y_seq = create_sequences(combined_scaled_df, seq_length, 'target')
        if X_seq.shape[0] == 0:
            logger.error("Failed to create sequences for GRU")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        train_seq_len = len(train) - seq_length
        if train_seq_len <= 0:
            logger.error(f"Train sequence length too short: {train_seq_len}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        X_train_seq, y_train_seq = X_seq[:train_seq_len], y_seq[:train_seq_len]
        X_test_seq, y_test_seq = X_seq[train_seq_len:], y_seq[train_seq_len:]
        logger.info(f"GRU: X_train_seq shape {X_train_seq.shape}, X_test_seq shape {X_test_seq.shape}")
        
        model = Sequential([
            GRU(32, activation='relu', input_shape=(seq_length, 1)),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_seq, y_train_seq, epochs=CONFIG['epochs'], batch_size=CONFIG['batch_size'], verbose=0)
        
        # Recursive forecasting
        forecast = []
        input_seq = X_seq[train_seq_len - 1].copy()
        for i in range(CONFIG['forecast_horizon']):
            input_seq_reshaped = input_seq.reshape((1, seq_length, 1))
            pred = model.predict(input_seq_reshaped, verbose=0)[0, 0]
            forecast.append(pred)
            input_seq = np.roll(input_seq, -1, axis=0)
            input_seq[-1] = pred
        
        forecast_unscaled = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()
        residuals = train.iloc[seq_length:] - scaler.inverse_transform(
            model.predict(X_train_seq)[:, 0].reshape(-1, 1)
        ).flatten()
        forecast = pd.Series(forecast_unscaled, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"GRU forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"GRU: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error GRU: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_voting_ensemble(forecasts, test, forecast_index):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        valid_forecasts = [f for f in forecasts.values() if f is not None and not f.isna().any() and len(f) == CONFIG['forecast_horizon']]
        if len(valid_forecasts) < 2:
            logger.error(f"Not enough valid forecasts for Voting Ensemble: {len(valid_forecasts)}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        logger.info(f"Voting Ensemble using {len(valid_forecasts)} forecasts: {[k for k, f in forecasts.items() if f is not None and not f.isna().any() and len(f) == CONFIG['forecast_horizon']]}")
        forecast = pd.DataFrame(valid_forecasts).mean().values
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"Voting Ensemble forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        residuals = None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"Voting Ensemble: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error Voting Ensemble: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_weighted_ensemble(forecasts, test, forecast_index, metrics):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        valid_forecasts = {k: f for k, f in forecasts.items() if f is not None and not f.isna().any() and len(f) == CONFIG['forecast_horizon']}
        valid_metrics = {k: m['RMSE'] for k, m in metrics.items() if m['RMSE'] is not np.nan}
        valid_forecasts = {k: f for k, f in valid_forecasts.items() if k in valid_metrics}
        if len(valid_forecasts) < 2:
            logger.error(f"Not enough valid forecasts for Weighted Ensemble: {len(valid_forecasts)}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        logger.info(f"Weighted Ensemble using {len(valid_forecasts)} forecasts: {list(valid_forecasts.keys())}")
        weights = {k: 1 / max(m, 1e-8) for k, m in valid_metrics.items()}
        total_weight = sum(weights.values())
        weights = {k: w / total_weight for k, w in weights.items()}
        
        forecast = sum(weights[k] * f for k, f in valid_forecasts.items())
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"Weighted Ensemble forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        residuals = None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"Weighted Ensemble: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error Weighted Ensemble: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

def run_stacking_ensemble(train, test, forecast_index, forecasts_train, forecasts_test):
    logger = logging.getLogger(__name__)
    start_time = time.time()
    try:
        X_train = pd.DataFrame({k: f for k, f in forecasts_train.items() if f is not None and not f.isna().any()})
        X_test = pd.DataFrame({k: f for k, f in forecasts_test.items() if f is not None and not f.isna().any() and len(f) == CONFIG['forecast_horizon']})
        if X_train.empty or X_test.empty or len(X_train.columns) < 2:
            logger.error(f"Not enough valid forecasts for Stacking Ensemble: {len(X_train.columns)}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        
        logger.info(f"Stacking Ensemble using {len(X_train.columns)} forecasts: {list(X_train.columns)}")
        if X_train.isna().any().any() or X_test.isna().any().any():
            logger.warning("NaN in forecasts_train or forecasts_test, filling...")
            X_train = X_train.fillna(X_train.mean(numeric_only=True))
            X_test = X_test.fillna(X_test.mean(numeric_only=True))
        
        model = LinearRegression()
        model.fit(X_train, train)
        forecast = model.predict(X_test)
        residuals = train - model.predict(X_train)
        forecast = pd.Series(forecast, index=forecast_index)
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"Stacking Ensemble forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None
        metrics = calculate_metrics(test, forecast)
        logger.info(f"Stacking Ensemble: RMSE={metrics[0]:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, metrics, None
    except Exception as e:
        logger.error(f"Error Stacking Ensemble: {str(e)}")
        return None, None, (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan), None

# Model Execution Pipeline
def run_model_for_target(target, train, test, forecast_index, model_name, model_func, params, exog_cols, config=CONFIG):
    logger = logging.getLogger(__name__)
    logger.info(f"Running {model_name} for {target}")
    start_time = time.time()
    try:
        if model_name in ['XGBoost', 'LightGBM', 'SVR']:
            feature_cols = [c for c in train.columns if c != target and c in test.columns]
            if not feature_cols:
                logger.error(f"No valid features for {model_name}")
                return None
            train_X = train[feature_cols]
            test_X = test[feature_cols]
            logger.info(f"{model_name}: train_X shape {train_X.shape}, test_X shape {test_X.shape}")
            forecast, residuals, metrics, ci = model_func(train[target], test[target], forecast_index, train_X, test_X)
        elif model_name in ['LSTM', 'GRU']:
            # Pass train_X, test_X for compatibility, but they are not used
            train_X = train[[target]]
            test_X = test[[target]]
            logger.info(f"{model_name}: train_X shape {train_X.shape}, test_X shape {test_X.shape}")
            forecast, residuals, metrics, ci = model_func(train[target], test[target], forecast_index, train_X, test_X)
        elif model_name in ['SARIMA', 'Prophet']:
            exog_train = train[exog_cols] if exog_cols else None
            exog_test = test[exog_cols] if exog_cols else None
            forecast, residuals, metrics, ci = model_func(train[target], test[target], forecast_index, exog_train, exog_test)
        elif model_name in ['Voting Ensemble', 'Weighted Ensemble']:
            forecast, residuals, metrics, ci = model_func(params['forecasts'], test[target], forecast_index, params.get('metrics', {}))
        elif model_name == 'Stacking Ensemble':
            forecast, residuals, metrics, ci = model_func(train[target], test[target], forecast_index, 
                                                        params['forecasts_train'], params['forecasts_test'])
        else:
            forecast, residuals, metrics, ci = model_func(train[target], test[target], forecast_index)
        
        if forecast is None or pd.isna(metrics[0]):
            logger.error(f"{model_name} for {target} failed")
            return None
        
        if len(forecast) != CONFIG['forecast_horizon']:
            logger.error(f"{model_name} forecast length {len(forecast)} != {CONFIG['forecast_horizon']}")
            return None
        
        plot_forecast(
            train[target][-36:], test[target], forecast, forecast_index,
            f'{model_name} Forecast for {target}', target, f'{target}_{model_name}_forecast.png', ci
        )
        if residuals is not None and len(residuals) > 1:
            plot_residual_acf(
                residuals.dropna(), f'ACF of Residuals - {model_name} ({target})',
                f'{target}_{model_name}_acf.png'
            )
        
        logger.info(f"Completed {model_name} for {target} in {time.time() - start_time:.2f}s")
        return {
            'Target': target,
            'Model': model_name,
            'RMSE': metrics[0],
            'MAE': metrics[1],
            'MAPE': metrics[2],
            'sMAPE': metrics[3],
            'NormMAPE': metrics[4],
            'DirAcc': metrics[5],
            'Forecast': forecast,
            'Residuals': residuals,
            'CI': ci
        }
    except Exception as e:
        logger.error(f"Error running {model_name} for {target}: {str(e)}")
        return None

# Main Execution
def main():
    logger = setup_logging(CONFIG['img_dir'])
    start_time = time.time()
    try:
        # Load và tiền xử lý dữ liệu
        cpi_data = pd.read_csv('data/cpi.csv', usecols=['t', 'cpi'])
        cpi_data.rename(columns={'t': 'date'}, inplace=True)
        cpi_data['time'] = pd.to_datetime(cpi_data['date'], format='%b-%y')
        cpi_data.set_index('time', inplace=True)
        cpi_data['cpi_mom'] = cpi_data['cpi'].pct_change() * 100
        cpi_data = cpi_data[['cpi_mom']].dropna()
        logger.info(f"CPI data shape: {cpi_data.shape}")
        if cpi_data.isna().any().any() or np.isinf(cpi_data).any().any():
            logger.warning("NaN/Inf in cpi_data, filling...")
            cpi_data = cpi_data.fillna(method='ffill').fillna(cpi_data.mean(numeric_only=True))

        exog_data = pd.read_csv('data/exog_data.csv')
        exog_data['Ngày'] = pd.to_datetime(exog_data['Ngày'])
        exog_data.set_index('Ngày', inplace=True)
        exog_cols = ['oil_price', 'gold_price', 'interest_rate']
        logger.info(f"Exogenous data shape: {exog_data.shape}")
        if exog_data.isna().any().any() or np.isinf(exog_data).any().any():
            logger.warning("NaN/Inf in exog_data, filling...")
            exog_data = exog_data.fillna(method='ffill').fillna(exog_data.mean(numeric_only=True))

        # Join và xử lý NaN
        data = cpi_data.join(exog_data, how='inner')
        logger.info(f"Joined data shape: {data.shape}")
        if data.isna().any().any() or np.isinf(data).any().any():
            logger.warning("NaN/Inf in joined data, filling...")
            data = data.fillna(method='ffill').fillna(data.mean(numeric_only=True))

        # Kiểm tra đủ dữ liệu cho dự báo
        if len(data) < CONFIG['min_data_length'] + CONFIG['forecast_horizon']:
            raise ValueError(f"Data too short: {len(data)} < {CONFIG['min_data_length'] + CONFIG['forecast_horizon']}")

        # Tạo features
        data_features = create_features(data, 'cpi_mom', exog_cols)
        logger.info(f"Features shape: {data_features.shape}")
        if data_features.isna().any().any() or np.isinf(data_features).any().any():
            logger.warning("NaN/Inf in data_features, filling...")
            data_features = data_features.fillna(data_features.mean(numeric_only=True))
        
        # Split data
        train_size = len(data) - CONFIG['forecast_horizon']
        train, test = data_features[:train_size], data_features[train_size:]
        if len(test) != CONFIG['forecast_horizon']:
            raise ValueError(f"Test set size {len(test)} != forecast_horizon {CONFIG['forecast_horizon']}")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")
        
        forecast_index = pd.date_range(start=test.index[0], periods=CONFIG['forecast_horizon'], freq='MS')
        logger.info(f"Forecast index: {forecast_index[0]} to {forecast_index[-1]}")
        
        # Plot decomposition
        plot_decomposition(data['cpi_mom'], period=CONFIG['seasonal_periods'], 
                         filename='cpi_mom_decomposition.png')
        
        # Define models
        models = {
            'Exponential Smoothing': (run_exponential_smoothing, {}),
            'SARIMA': (run_sarima, {}),
            'Prophet': (run_prophet, {}),
            'XGBoost': (run_xgboost, {}),
            'LightGBM': (run_lightgbm, {}),
            'SVR': (run_svr, {}),
            'LSTM': (run_lstm, {}),
            'GRU': (run_gru, {})
        }
        
        # Run base models in parallel
        results = []
        forecasts_mom = {}
        metrics_mom = {}
        forecasts_train = {}
        forecasts_test = {}
        logger.info("Running base models for cpi_mom")
        
        tasks = [
            delayed(run_model_for_target)(
                'cpi_mom', train, test, forecast_index, model_name, model_func, params, exog_cols
            ) for model_name, (model_func, params) in models.items()
        ]
        model_results = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1, backend='loky')(tasks)
        
        # Process base model results
        for result in model_results:
            if result is not None:
                results.append({
                    'Target': result['Target'],
                    'Model': result['Model'],
                    'RMSE': result['RMSE'],
                    'MAE': result['MAE'],
                    'MAPE': result['MAPE'],
                    'sMAPE': result['sMAPE'],
                    'NormMAPE': result['NormMAPE'],
                    'DirAcc': result['DirAcc']
                })
                forecasts_mom[result['Model']] = result['Forecast']
                metrics_mom[result['Model']] = {'RMSE': result['RMSE']}
                forecasts_train[result['Model']] = train['cpi_mom'] - result['Residuals'] if result['Residuals'] is not None else None
                forecasts_test[result['Model']] = result['Forecast']
        
        # Run ensemble models
        ensemble_models = {
            'Voting Ensemble': (run_voting_ensemble, {'forecasts': forecasts_mom}),
            'Weighted Ensemble': (run_weighted_ensemble, {'forecasts': forecasts_mom, 'metrics': metrics_mom}),
            'Stacking Ensemble': (run_stacking_ensemble, {'forecasts_train': forecasts_train, 'forecasts_test': forecasts_test})
        }
        
        tasks = [
            delayed(run_model_for_target)(
                'cpi_mom', train, test, forecast_index, model_name, model_func, params, exog_cols
            ) for model_name, (model_func, params) in ensemble_models.items()
        ]
        ensemble_results = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1, backend='loky')(tasks)
        
        # Process ensemble results
        for result in ensemble_results:
            if result is not None:
                results.append({
                    'Target': result['Target'],
                    'Model': result['Model'],
                    'RMSE': result['RMSE'],
                    'MAE': result['MAE'],
                    'MAPE': result['MAPE'],
                    'sMAPE': result['sMAPE'],
                    'NormMAPE': result['NormMAPE'],
                    'DirAcc': result['DirAcc']
                })
                forecasts_mom[result['Model']] = result['Forecast']
                metrics_mom[result['Model']] = {'RMSE': result['RMSE']}
        
        # Generate comparison plots and save results
        if forecasts_mom:
            plot_comparison_forecasts(
                train['cpi_mom'][-36:], test['cpi_mom'], forecasts_mom, forecast_index,
                'Comparison of Forecasts for cpi_mom', 'cpi_mom', 
                'cpi_mom_model_comparison.png', metrics=metrics_mom
            )
        
        results_df = pd.DataFrame(results)
        if not results_df.empty:
            print(results_df)
            results_df.to_csv(CONFIG['results_file'], index=False)
            logger.info(f"Results saved to {CONFIG['results_file']}")
            plot_metrics_bar(results_df, 'cpi_mom_metrics_comparison.png')
        
        if forecasts_mom:
            combined_forecast = pd.DataFrame({'Date': forecast_index})
            for model_name, forecast in forecasts_mom.items():
                combined_forecast[f'{model_name}_cpi_mom'] = forecast
            combined_forecast.to_csv(f'{CONFIG["img_dir"]}/combined_forecast_cpi_mom.csv', index=False)
            logger.info(f"Combined forecasts saved to {CONFIG['img_dir']}/combined_forecast_cpi_mom.csv")
        
        logger.info(f"Total execution time: {time.time() - start_time:.2f}s")
            
    except Exception as e:
        logger.error(f"Main program error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

c:\Users\ASUS\anaconda3\envs\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 8 out of 8 | elapsed: 11.5min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 1 out of 1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done 3 out of 3 | elapsed:    0.6s finished


    Target                  Model      RMSE       MAE        MAPE       sMAPE  \
0  cpi_mom  Exponential Smoothing  0.319763  0.285126  256.581561  140.962257   
1  cpi_mom                 SARIMA  0.325483  0.286272  252.902619  137.211988   
2  cpi_mom                Prophet  0.300574  0.250898  231.055562  119.815358   
3  cpi_mom                XGBoost  0.291215  0.258049  221.849376  132.195043   
4  cpi_mom               LightGBM  0.261888  0.205114   93.970002   97.258134   
5  cpi_mom                    SVR  0.194838  0.143999  126.863913   80.239780   
6  cpi_mom                   LSTM  0.476645  0.353113  115.392848  169.933539   
7  cpi_mom                    GRU  0.499680  0.405019  196.234463  170.683812   
8  cpi_mom      Weighted Ensemble  0.236918  0.198847  101.444213  121.136437   

     NormMAPE     DirAcc  
0  752.574411  36.363636  
1  741.783777  36.363636  
2  677.704598  54.545455  
3  650.702111  72.727273  
4  275.621595  81.818182  
5  372.102087  81.818182  
